In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# constants
ROOT_PATH = 'C:/AgodaGit/passivbot' 
DOWNLOADED_CONFIG_PATH = ROOT_PATH + '/downloaded_configs/passivbot_v5.9.x'

In [15]:
import pandas as pd

pbos = pd.read_csv(ROOT_PATH + '/downloaded_configs/pbos/tedy_best_finding_bt_2020-01-01_2023-05-20_1000_1_XRPUSDT_LTCUSDT_ADAUSDT_DOTUSDT_UNIUSDT_DOGEUSDT_MATICUSDT_BNBUSDT_SOLUSDT_TRXUSDT_AVAXUSDT.csv')

pbos

,Unnamed: 0,strat,gs,s_f_equ_long,s_loss,low_equ_bal,pa_dist_mean_long,l_we,adg_exposure,avg_max_stuck,avg_hrs_stuck_avg,pa_distance_max_long,most_loss
0,3036,01198,-1.0,430079.884457,0.0,0.143494,2.030996,1.0,0.369580,3513.856061,0.227275,8.927336,3584.373426
1,5482,3e028,-1.0,523856.663078,0.0,0.145238,1.995830,1.0,0.365476,3497.381818,0.288658,8.854136,3871.101280
2,5049,0574c,-1.0,201296.950709,0.0,0.141098,2.103764,1.0,0.357568,3756.518182,0.297472,9.186677,4732.116420
3,8593,2bed1,-1.0,277098.109123,0.0,0.142115,2.018434,1.0,0.357175,3388.671212,0.374670,8.917022,4969.069633
4,4527,08179,-1.0,193778.443593,0.0,0.139113,2.112440,1.0,0.354933,3772.989394,0.330920,9.204810,3462.429608
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,8247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9615,8316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9616,8803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9617,8852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:

import glob
import os

# list all json files in cfgs_bkup and its child folders
live_configs = glob.glob(ROOT_PATH + '/cfgs_bkup/**/*.json', recursive=True)
# normalize json_files by replacing all \\ with /
live_configs = [f.replace('\\', '/') for f in live_configs]


#group json_files by immediate parent folder
config_by_version = {}
for f in live_configs:
    folder = os.path.dirname(f).split('/')[-1]
    # folder looks like this : 5_9_0 or 5_9_3 or 5_9_4 etc. convert it to v5.9.0 or v.5.9.3 or v5.9.4
    folder = 'v' + folder.replace('_', '.')
    if folder not in config_by_version:
        config_by_version[folder] = {'live_configs': [] }
    config_by_version[folder]['live_configs'].append({
        'file': f,
        'coin': os.path.basename(f).replace('.json', '')
    })

config_by_version   


{'v5.9.0': {'live_configs': [{'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/ATOM.json',
    'coin': 'ATOM'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/CHZ.json', 'coin': 'CHZ'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DOGE.json', 'coin': 'DOGE'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DOT.json', 'coin': 'DOT'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DYDX.json', 'coin': 'DYDX'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/EOS.json', 'coin': 'EOS'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/HBAR.json', 'coin': 'HBAR'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/LINK.json', 'coin': 'LINK'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/MANA.json', 'coin': 'MANA'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/MATIC.json',
    'coin': 'MATIC'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/OP.json', 'coin': 'OP'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/SAND.json', 'coin': 'SAND'},
   {'file': 'C

In [17]:
# now find all config.json files in downloaded_configs/passivbot_v5.9.x folder
downloaded_configs = glob.glob(DOWNLOADED_CONFIG_PATH + '/**/config.json', recursive=True)
# normalize json_files by replacing all \\ with /
downloaded_configs = [f.replace('\\', '/') for f in downloaded_configs]


In [18]:
import re
'''
Result file looks like this:
    +------------------------------------------+
    |                 Summary                  |
    +-----------------------------+------------+
    | Metric                      | Value      |
    +-----------------------------+------------+
    | Exchange                    | bybit      |
    | Market type                 | futures    |
    | Symbol                      | ADAUSDT    |
    | Passivbot mode              | emas       |
    | No. days                    | 1100.0     |
    +-----------------------------+------------+
Write a parser function which takes in the path to result.txt file, reads it and then split the Metric and Value columns and put them in a dictionary and return it
'''
def parse_result_file(result_file):
    with open(result_file, 'r') as file:

        pattern = r'/v(\d+\.\d+(?:\.\w+)?)/'
        match = re.search(pattern, result_file)
        if match:
            version = match.group(1)
        else:
            version = None
            
        
        data = file.read().split('\n')
        d = {'version':version}
        current_state = None
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()
                # dont add if key is empty string
                if key == '':
                    continue

                if key == 'Long':
                    current_state = 'L'
                if key == 'Short':
                    current_state = 'S'
                
                # check if value can be converted to float and if so, convert it
                try:
                    value = value.replace('%', '').strip()
                    value = float(value)
                except:
                    pass
                   
                # if current_state is not None, then add it to key with a prefix of (L) or (S)
                if current_state is not None:
                    key = f'({current_state}){key}'
                d[key] = value
    return d

# test it
parse_result_file(r'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/v5.9.5/recursive/july_2023/recursive_july_03_auenabled_0.2WE/RENUSDT_20230714162727_dfc69/RENUSDT/result.txt')

{'version': '5.9.5',
 'Metric': 'Value',
 'Exchange': 'bybit',
 'Market type': 'futures',
 'Symbol': 'RENUSDT',
 'Passivbot mode': 'recursive_grid',
 'ADG n subdivisions': 10.0,
 'No. days': 560.0,
 'Starting balance': 1000.0,
 '(L)Long': 'True',
 '(L)ADG per exposure': 0.0545,
 '(L)ADG weighted per exposure': 0.0796,
 '(L)Final balance': 1063.5,
 '(L)Final equity': 1063.5,
 '(L)Net PNL + fees': 63.5003,
 '(L)Net Total gain': 6.35,
 '(L)Average daily gain': 0.0109,
 '(L)Average daily gain weighted': 0.0159,
 '(L)Loss to profit ratio': 0.00014,
 '(L)Exposure ratios mean': 0.0084203,
 '(L)Price action distance mean': 0.0108677,
 '(L)Price action distance std': 0.00834529,
 '(L)Price action distance max': 0.0491622,
 '(L)Closest bankruptcy': 100.0,
 '(L)Lowest equity/balance ratio': 0.9737,
 '(L)Mean of 10 worst eq/bal ratios': 0.9921,
 '(L)Equity/balance ratio std': 0.0003235,
 '(L)No. fills': 17730.0,
 '(L)No. entries': 5623.0,
 '(L)No. closes': 12107.0,
 '(L)No. initial entries': 4623.

In [19]:
import json

_l_config = set()
# now iterate over config_by_version and find the downloaded config.json file for each live config
for version in config_by_version:
    for live_config in config_by_version[version]['live_configs']:
        coin = live_config['coin']+'USDT'
        file_content = json.load(open(live_config['file']))
        # find the list of matches in downloaded config.json files for this coin where version must be in the path and coin must be in the path
        matches = [f for f in downloaded_configs if version in f and coin in f]

        match_found = None
        #now find the exact match where the json.load of config.json is the exact same json as file_content
        for match in matches:
            if file_content == json.load(open(match)):
                match_found = match
                break
        if match_found is None:
            print(f'No match found for {coin} in {version}')
        else:
            _l_config.add(match_found)
            live_config['downloaded_config'] = match_found
            # find the backtest result file ( called result.txt ) for this downloaded config.json file. it is found in a subdirectory within the same folder as config.json
            #print(glob.glob(os.path.dirname(match_found)))
            result_file = glob.glob(os.path.dirname(match_found) + '/**/result.txt', recursive=True)
            live_config['result_file'] = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
            live_config['result'] = parse_result_file(live_config['result_file']) if live_config['result_file'] is not None else None
            live_config['result']['version'] = version
            
config_by_version   
        

{'v5.9.0': {'live_configs': [{'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/ATOM.json',
    'coin': 'ATOM',
    'downloaded_config': 'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/recursive/longshort/harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off/ATOMUSDT_20221222000537_d44da/config.json',
    'result_file': 'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/recursive/longshort/harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off/ATOMUSDT_20221222000537_d44da/ATOMUSDT/result.txt',
    'result': {'version': 'v5.9.0',
     'Metric': 'Value',
     'Exchange': 'binance',
     'Market type': 'futures',
     'Symbol': 'ATOMUSDT',
     'No. days': 1019.49,
     'Starting balance': 1500.0,
     '(L)Long': 'True',
     '(L)ADG realized per exposure': 0.271,
     '(L)Final balance': 2995.62,
     '(L)Final equity': 2995.63,
     '(L)Net PNL + fees': 1495.62,
     '(L)Total gain': 101.8,
 

In [20]:
# iterate over downloaded_configs, find result.txt file and parse it and add result to a pandas dataframe
import pandas as pd
result_analysis = pd.DataFrame()
for downloaded_config in downloaded_configs:     
    result_file = glob.glob(os.path.dirname(downloaded_config) + '/**/result.txt', recursive=True)
    result_file = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
    if result_file is not None:
        result = parse_result_file(result_file)
        if result['version'] is None:
            print(f'No version found for {result_file}')
        result['file'] = downloaded_config
        if downloaded_config not in _l_config:
            result['live'] = False
        else:
            result['live'] = True
        # convert result to a pandas dataframe and append it to result_analysis dataframe
        result = pd.DataFrame(result, index=[0])
        result_analysis = pd.concat([result_analysis, result], ignore_index=True)

result_analysis

,version,Metric,Exchange,Market type,Symbol,No. days,Starting balance,(L)Long,(L)ADG realized per exposure,(L)Final balance,(L)Final equity,(L)Net PNL + fees,(L)Total gain,(L)Average daily gain,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Equity/balance ratio std,(L)No. fills,(L)No. entries,(L)No. closes,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck closes,(L)No. normal closes,(L)Average n fills per day,(L)Mean hours stuck,(L)Max hours stuck,(L)PNL sum,(L)Profit sum,(L)Loss sum,(L)Fee sum,(L)Biggest pos size,(L)Biggest pos cost,(L)Volume quote,file,live,(S)Short,(S)ADG realized per exposure,(S)Final balance,(S)Final equity,(S)Net PNL + fees,(S)Total gain,(S)Average daily gain,(S)Loss to profit ratio,(S)Price action distance mean,(S)Price action distance std,(S)Price action distance max,(S)Closest bankruptcy,(S)Lowest equity/balance ratio,(S)Equity/balance ratio std,(S)No. fills,(S)No. entries,(S)No. closes,(S)No. initial entries,(S)No. reentries,(S)No. unstuck entries,(S)No. unstuck closes,(S)No. normal closes,(S)Average n fills per day,(S)Mean hours stuck,(S)Max hours stuck,(S)PNL sum,(S)Profit sum,(S)Loss sum,(S)Fee sum,(S)Biggest pos size,(S)Biggest pos cost,(S)Volume quote,(L)DG mean std ratio,(S)DG mean std ratio,Passivbot mode,(L)No. unstuck/EMA entries,(L)No. unstuck/EMA closes,(S)No. unstuck/EMA entries,(S)No. unstuck/EMA closes,ADG n subdivisions,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Net Total gain,(L)Average daily gain weighted,(L)Mean of 10 worst eq/bal ratios,(L)Exposure ratios mean,(S)ADG per exposure,(S)ADG weighted per exposure,(S)Net Total gain,(S)Average daily gain weighted,(S)Mean of 10 worst eq/bal ratios
0,5.9.0,Value,binance,futures,ATOMUSDT,1046.000,1500.0,True,0.0954,4068.83,4069.30,2568.83,175.1,0.1290,0.54090,0.018617,0.030567,1.535200,99.74,0.3945,0.021970,28358.0,5655.0,22703.0,1992.0,3631.0,32.0,645.0,22058.0,27.10,1.381770,149.2500,2626.0,5719.0,-3093.00,-56.99,582.6,5005.0,569909.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/ATOMUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.9.0,Value,binance,futures,CHZUSDT,705.708,1500.0,True,0.2780,10654.30,5267.04,9154.32,620.8,0.2660,0.11700,0.087484,0.162400,2.036520,99.81,0.3294,0.104300,38468.0,19179.0,19289.0,2272.0,16898.0,9.0,1507.0,17782.0,54.50,0.674136,895.3000,9312.0,10550.0,-1233.00,-157.50,84720.0,10650.0,1574730.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/CHZUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.9.0,Value,binance,futures,DOGEUSDT,900.625,1500.0,True,0.4130,61566.10,61566.30,60066.10,4185.0,0.4240,0.04594,0.007174,0.010823,0.684707,99.99,0.5936,0.008557,252063.0,30805.0,221258.0,6493.0,24189.0,123.0,13974.0,207284.0,280.00,0.147534,109.6670,62780.0,65800.0,-3023.00,-2715.00,831775.0,60420.0,27145100.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOGEUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.9.0,Value,binance,futures,DOTUSDT,857.708,15

In [21]:
# list all columns in result_analysis, one column per line , wrapped in single quotes and separated by comma
print('\n'.join([f"'{c}'," for c in result_analysis.columns]))

'version',
'Metric',
'Exchange',
'Market type',
'Symbol',
'No. days',
'Starting balance',
'(L)Long',
'(L)ADG realized per exposure',
'(L)Final balance',
'(L)Final equity',
'(L)Net PNL + fees',
'(L)Total gain',
'(L)Average daily gain',
'(L)Loss to profit ratio',
'(L)Price action distance mean',
'(L)Price action distance std',
'(L)Price action distance max',
'(L)Closest bankruptcy',
'(L)Lowest equity/balance ratio',
'(L)Equity/balance ratio std',
'(L)No. fills',
'(L)No. entries',
'(L)No. closes',
'(L)No. initial entries',
'(L)No. reentries',
'(L)No. unstuck entries',
'(L)No. unstuck closes',
'(L)No. normal closes',
'(L)Average n fills per day',
'(L)Mean hours stuck',
'(L)Max hours stuck',
'(L)PNL sum',
'(L)Profit sum',
'(L)Loss sum',
'(L)Fee sum',
'(L)Biggest pos size',
'(L)Biggest pos cost',
'(L)Volume quote',
'file',
'live',
'(S)Short',
'(S)ADG realized per exposure',
'(S)Final balance',
'(S)Final equity',
'(S)Net PNL + fees',
'(S)Total gain',
'(S)Average daily gain',
'(S)Loss to profi

In [22]:
import requests

def get_minimum_qty(symbol):
    api_url = f"https://fapi.binance.com/fapi/v1/exchangeInfo"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        for symbol_info in data['symbols']:
            if symbol_info['symbol'] == symbol:
                return float(symbol_info['filters'][1]['stepSize'])
        return None
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

symbol = "BTCUSDT"  # Replace with the symbol you're interested in
minimum_qty = get_minimum_qty(symbol)

if minimum_qty is not None:
    print(f"Minimum quantity for {symbol}: {minimum_qty}")
else:
    print(f"Failed to get minimum quantity for {symbol}")


Minimum quantity for BTCUSDT: 0.001


In [23]:

import requests

def get_24h_trading_volume(symbol):
    base_url = "https://api.binance.com/api/v3/ticker/24hr"
    params = {"symbol": symbol}

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        trading_volume = round(float(data["quoteVolume"]) / 10**6 , 2 )
        return trading_volume
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        return None
    
x = get_24h_trading_volume('BTCUSDT')

x

1766.67

In [24]:

data_url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
data = requests.get(data_url).json()
prices_url = "https://fapi.binance.com/fapi/v1/ticker/price"
pricedata = requests.get(prices_url).json()

# get all symbols from Symbol column in result_analysis
symbols = result_analysis['Symbol'].unique().tolist()

notional_volume = pd.DataFrame(columns=["Symbol", "min_notional", "volume"])

try:
    for datas in data["symbols"]:
        if "USDT" in datas["pair"] and datas["pair"] in symbols:
            symbol = datas["pair"]
            min_qty = float(datas["filters"][1]["minQty"] )
            min_notional_fixed =  float(datas["filters"][5]["notional"])
    
            for i in pricedata:
                if i["symbol"] == symbol:
                    price = i["price"]
    
            min_notional_calc = min_qty * float(price)

            if min_notional_calc <= min_notional_fixed:
                min_notional = min_notional_fixed
            else:
                min_notional = min_notional_calc
            
            volume_in_million_usd = get_24h_trading_volume(symbol)
            notional_volume = pd.concat([notional_volume, pd.DataFrame({"Symbol": [symbol], "min_notional": [min_notional], "volume": [volume_in_million_usd]})], ignore_index=True)



except Exception as e:
    print(f'Error: {e}')
    pass


notional_volume

,Symbol,min_notional,volume
0,AUDIOUSDT,5.0,2.05
1,CHRUSDT,5.0,2.42
2,CTSIUSDT,5.0,1.57
3,OMGUSDT,5.0,1.13
4,GALAUSDT,5.0,17.37
...,...,...,...
83,FTMUSDT,5.0,16.94
84,KAVAUSDT,5.0,6.13
85,GALUSDT,5.0,3.5
86,BTCUSDT,26.0944,1766.86


In [25]:
# join result_analysis with notional_volume on Symbol column
result_analysis = result_analysis.merge(notional_volume, on='Symbol', how='left')

result_analysis

,version,Metric,Exchange,Market type,Symbol,No. days,Starting balance,(L)Long,(L)ADG realized per exposure,(L)Final balance,(L)Final equity,(L)Net PNL + fees,(L)Total gain,(L)Average daily gain,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Equity/balance ratio std,(L)No. fills,(L)No. entries,(L)No. closes,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck closes,(L)No. normal closes,(L)Average n fills per day,(L)Mean hours stuck,(L)Max hours stuck,(L)PNL sum,(L)Profit sum,(L)Loss sum,(L)Fee sum,(L)Biggest pos size,(L)Biggest pos cost,(L)Volume quote,file,live,(S)Short,(S)ADG realized per exposure,(S)Final balance,(S)Final equity,(S)Net PNL + fees,(S)Total gain,(S)Average daily gain,(S)Loss to profit ratio,(S)Price action distance mean,(S)Price action distance std,(S)Price action distance max,(S)Closest bankruptcy,(S)Lowest equity/balance ratio,(S)Equity/balance ratio std,(S)No. fills,(S)No. entries,(S)No. closes,(S)No. initial entries,(S)No. reentries,(S)No. unstuck entries,(S)No. unstuck closes,(S)No. normal closes,(S)Average n fills per day,(S)Mean hours stuck,(S)Max hours stuck,(S)PNL sum,(S)Profit sum,(S)Loss sum,(S)Fee sum,(S)Biggest pos size,(S)Biggest pos cost,(S)Volume quote,(L)DG mean std ratio,(S)DG mean std ratio,Passivbot mode,(L)No. unstuck/EMA entries,(L)No. unstuck/EMA closes,(S)No. unstuck/EMA entries,(S)No. unstuck/EMA closes,ADG n subdivisions,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Net Total gain,(L)Average daily gain weighted,(L)Mean of 10 worst eq/bal ratios,(L)Exposure ratios mean,(S)ADG per exposure,(S)ADG weighted per exposure,(S)Net Total gain,(S)Average daily gain weighted,(S)Mean of 10 worst eq/bal ratios,min_notional,volume
0,5.9.0,Value,binance,futures,ATOMUSDT,1046.000,1500.0,True,0.0954,4068.83,4069.30,2568.83,175.1,0.1290,0.54090,0.018617,0.030567,1.535200,99.74,0.3945,0.021970,28358.0,5655.0,22703.0,1992.0,3631.0,32.0,645.0,22058.0,27.10,1.381770,149.2500,2626.0,5719.0,-3093.00,-56.99,582.6,5005.0,569909.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/ATOMUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,18.81
1,5.9.0,Value,binance,futures,CHZUSDT,705.708,1500.0,True,0.2780,10654.30,5267.04,9154.32,620.8,0.2660,0.11700,0.087484,0.162400,2.036520,99.81,0.3294,0.104300,38468.0,19179.0,19289.0,2272.0,16898.0,9.0,1507.0,17782.0,54.50,0.674136,895.3000,9312.0,10550.0,-1233.00,-157.50,84720.0,10650.0,1574730.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/CHZUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,8.77
2,5.9.0,Value,binance,futures,DOGEUSDT,900.625,1500.0,True,0.4130,61566.10,61566.30,60066.10,4185.0,0.4240,0.04594,0.007174,0.010823,0.684707,99.99,0.5936,0.008557,252063.0,30805.0,221258.0,6493.0,24189.0,123.0,13974.0,207284.0,280.00,0.147534,109.6670,62780.0,65800.0,-3023.00,-2715.00,831775.0,60420.0,27145100.0,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOGEUSDT_20230101015912_38fd6/config.json,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,76.51

In [26]:
# allowed_symbols = [
#     "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
#     "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
#     "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
#     "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
#     "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
#     "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
#     "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
# ]
# # drop rows from result_analysis where symbol is not in allowed_symbols
# result_analysis_filtered = result_analysis[result_analysis['Symbol'].isin(allowed_symbols)]


result_analysis_filtered = result_analysis[[
    'Symbol',
    'Exchange',
    'version',
    'No. days',
    'Passivbot mode',
    'volume',
    'min_notional',
    'live',
    
    '(L)Long',
    '(L)ADG realized per exposure',
    '(L)ADG per exposure',
    '(L)ADG weighted per exposure',
    '(L)Average daily gain',
    '(L)Average daily gain weighted',
    '(L)DG mean std ratio',
    '(L)Exposure ratios mean',
    '(L)Loss to profit ratio',
    '(L)Price action distance mean',
    '(L)Price action distance std',
    '(L)Price action distance max',
    '(L)Closest bankruptcy',
    '(L)Lowest equity/balance ratio',
    '(L)Mean of 10 worst eq/bal ratios',
    '(L)Equity/balance ratio std',
    '(L)No. initial entries',
    '(L)No. reentries',
    '(L)No. unstuck entries',
    '(L)No. unstuck/EMA entries',
    '(L)No. unstuck closes',
    '(L)No. unstuck/EMA closes',
    '(L)No. normal closes',
    '(L)Mean hours stuck',
    '(L)Max hours stuck',


    '(S)Short',
    '(S)ADG realized per exposure',
    '(S)ADG per exposure',
    '(S)ADG weighted per exposure',
    '(S)Average daily gain',
    '(S)Average daily gain weighted',
    '(S)DG mean std ratio',
    '(S)Loss to profit ratio',
    '(S)Price action distance mean',
    '(S)Price action distance std',
    '(S)Price action distance max',
    '(S)Closest bankruptcy',
    '(S)Lowest equity/balance ratio',
    '(S)Mean of 10 worst eq/bal ratios',
    '(S)Equity/balance ratio std',
    '(S)No. initial entries',
    '(S)No. reentries',
    '(S)No. unstuck entries',
    '(S)No. unstuck/EMA entries',
    '(S)No. unstuck closes',
    '(S)No. unstuck/EMA closes',
    '(S)No. normal closes',
    '(S)Mean hours stuck',
    '(S)Max hours stuck',

    'file',

]]


# copy all values from  '(L)ADG realized per exposure' column to '(L)ADG per exposure' where '(L)ADG per exposure' is null
result_analysis_filtered.loc[result_analysis_filtered['(L)ADG per exposure'].isnull(), '(L)ADG per exposure'] = result_analysis_filtered['(L)ADG realized per exposure']
# do the same for 'S' columns
result_analysis_filtered.loc[result_analysis_filtered['(S)ADG per exposure'].isnull(), '(S)ADG per exposure'] = result_analysis_filtered['(S)ADG realized per exposure']

# drop columns  '(L)ADG realized per exposure' and '(S)ADG realized per exposure'
result_analysis_filtered.drop(['(L)ADG realized per exposure', '(S)ADG realized per exposure'], axis=1, inplace=True)

result_analysis_filtered

C:\Users\skhan\AppData\Local\Temp\ipykernel_29996\165143451.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_analysis_filtered.drop(['(L)ADG realized per exposure', '(S)ADG realized per exposure'], axis=1, inplace=True)


,Symbol,Exchange,version,No. days,Passivbot mode,volume,min_notional,live,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,(S)Short,(S)ADG per exposure,(S)ADG weighted per exposure,(S)Average daily gain,(S)Average daily gain weighted,(S)DG mean std ratio,(S)Loss to profit ratio,(S)Price action distance mean,(S)Price action distance std,(S)Price action distance max,(S)Closest bankruptcy,(S)Lowest equity/balance ratio,(S)Mean of 10 worst eq/bal ratios,(S)Equity/balance ratio std,(S)No. initial entries,(S)No. reentries,(S)No. unstuck entries,(S)No. unstuck/EMA entries,(S)No. unstuck closes,(S)No. unstuck/EMA closes,(S)No. normal closes,(S)Mean hours stuck,(S)Max hours stuck,file
0,ATOMUSDT,binance,5.9.0,1046.000,NaN,18.81,5.0,False,True,0.0954,NaN,0.1290,NaN,NaN,NaN,0.54090,0.018617,0.030567,1.535200,99.74,0.3945,NaN,0.021970,1992.0,3631.0,32.0,NaN,645.0,NaN,22058.0,1.381770,149.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/ATOMUSDT_20230101015912_38fd6/config.json
1,CHZUSDT,binance,5.9.0,705.708,NaN,8.77,5.0,False,True,0.2780,NaN,0.2660,NaN,NaN,NaN,0.11700,0.087484,0.162400,2.036520,99.81,0.3294,NaN,0.104300,2272.0,16898.0,9.0,NaN,1507.0,NaN,17782.0,0.674136,895.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/CHZUSDT_20230101015912_38fd6/config.json
2,DOGEUSDT,binance,5.9.0,900.625,NaN,76.51,5.0,False,True,0.4130,NaN,0.4240,NaN,NaN,NaN,0.04594,0.007174,0.010823,0.684707,99.99,0.5936,NaN,0.008557,6493.0,24189.0,123.0,NaN,13974.0,NaN,207284.0,0.147534,109.6670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOGEUSDT_20230101015912_38fd6/config.json
3,DOTUSDT,binance,5.9.0,857.708,NaN,11.91,5.0,False,True,0.2910,NaN,0.3000,NaN,NaN,NaN,0.05236,0.010547,0.017572,0.915960,99.77,0.5217,NaN,0.014570,4214.0,12789.0,108.0,NaN,142.0,NaN,64833.0,0.393519,265.3670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOTUSDT_20230101015912_38fd6/config.json
4,DYDXUSDT,binance,5.9.0,473.854,NaN,10.48,5.0,False,True,0.2640,NaN,0.2750,NaN,NaN,NaN,0.08731,0.017620,0.020605,0.386918,99.81,0.7211,NaN,0.015660,903.0,2124.0,25.0,NaN,187.0,NaN,14062.0,1.066940,99.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DYDXUSDT_20230101015912_38fd6/config.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,XRPUSDT,bybit,5.9.5,1200.000,recursive_grid,351.74,5.0,Fals

In [27]:
# drop Metric column and move file column to the beginning
result_analysis_filtered.to_csv(ROOT_PATH+'/analysis_scripts/result_analysis.csv')


In [28]:
# filter result_analysis_filtered where volume is greater > 5 and min_notional < 11
result_analysis_filtered = result_analysis_filtered[
    (result_analysis_filtered['volume'] > 5)
    & (result_analysis_filtered['min_notional'] < 11)]


result_analysis_filtered

,Symbol,Exchange,version,No. days,Passivbot mode,volume,min_notional,live,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,(S)Short,(S)ADG per exposure,(S)ADG weighted per exposure,(S)Average daily gain,(S)Average daily gain weighted,(S)DG mean std ratio,(S)Loss to profit ratio,(S)Price action distance mean,(S)Price action distance std,(S)Price action distance max,(S)Closest bankruptcy,(S)Lowest equity/balance ratio,(S)Mean of 10 worst eq/bal ratios,(S)Equity/balance ratio std,(S)No. initial entries,(S)No. reentries,(S)No. unstuck entries,(S)No. unstuck/EMA entries,(S)No. unstuck closes,(S)No. unstuck/EMA closes,(S)No. normal closes,(S)Mean hours stuck,(S)Max hours stuck,file
0,ATOMUSDT,binance,5.9.0,1046.000,NaN,18.81,5.0,False,True,0.0954,NaN,0.1290,NaN,NaN,NaN,0.54090,0.018617,0.030567,1.535200,99.74,0.3945,NaN,0.021970,1992.0,3631.0,32.0,NaN,645.0,NaN,22058.0,1.381770,149.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/ATOMUSDT_20230101015912_38fd6/config.json
1,CHZUSDT,binance,5.9.0,705.708,NaN,8.77,5.0,False,True,0.2780,NaN,0.2660,NaN,NaN,NaN,0.11700,0.087484,0.162400,2.036520,99.81,0.3294,NaN,0.104300,2272.0,16898.0,9.0,NaN,1507.0,NaN,17782.0,0.674136,895.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/CHZUSDT_20230101015912_38fd6/config.json
2,DOGEUSDT,binance,5.9.0,900.625,NaN,76.51,5.0,False,True,0.4130,NaN,0.4240,NaN,NaN,NaN,0.04594,0.007174,0.010823,0.684707,99.99,0.5936,NaN,0.008557,6493.0,24189.0,123.0,NaN,13974.0,NaN,207284.0,0.147534,109.6670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOGEUSDT_20230101015912_38fd6/config.json
3,DOTUSDT,binance,5.9.0,857.708,NaN,11.91,5.0,False,True,0.2910,NaN,0.3000,NaN,NaN,NaN,0.05236,0.010547,0.017572,0.915960,99.77,0.5217,NaN,0.014570,4214.0,12789.0,108.0,NaN,142.0,NaN,64833.0,0.393519,265.3670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DOTUSDT_20230101015912_38fd6/config.json
4,DYDXUSDT,binance,5.9.0,473.854,NaN,10.48,5.0,False,True,0.2640,NaN,0.2750,NaN,NaN,NaN,0.08731,0.017620,0.020605,0.386918,99.81,0.7211,NaN,0.015660,903.0,2124.0,25.0,NaN,187.0,NaN,14062.0,1.066940,99.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/neat/longonly/harmony_neat_auenabled_longonly_1.0WE_bwtpoff/DYDXUSDT_20230101015912_38fd6/config.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,SUSHIUSDT,bybit,5.9.5,990.000,recursive_grid,5.22,5.0,False

In [35]:
import warnings

# Filter and ignore FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)
# Set the display options to show all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
# Set the display option to show full content of cells
pd.set_option('display.max_colwidth', None)


'''
create a new dataset called result_analysis_long from result_analysis_filtered which contains :
- All the columns that starts with (L)
- And these columns :
    'Symbol',
    'Exchange',
    'version',
    'No. days',
    'Passivbot mode',
    'volume',
    'min_notional',
    'live',
'''
result_analysis_long = result_analysis_filtered[[c for c in result_analysis_filtered.columns if c.startswith('(L)')]]
result_analysis_long = result_analysis_long.join(result_analysis_filtered[[
    'Symbol',
    'Exchange',
    'version',
    'No. days',
    'Passivbot mode',
    'volume',
    'min_notional',
    'live',
    'file'
]])

# list all unique symbols in result_analysis_long
symbols = result_analysis_long['Symbol'].unique().tolist()
#display(symbols)
# for each symbol, find the best configuration
best_configs = {}
not_found_config = []
found_config = []
excel_data_to_write = {}
for symbol in symbols:
    # find all rows in result_analysis_long where Symbol is symbol
    df = result_analysis_long[result_analysis_long['Symbol'] == symbol]

    df = df[
        (df['(L)Closest bankruptcy'] > 99.0)
        & (df['(L)Closest bankruptcy'] > 99.0)
        & ((df['(L)Mean of 10 worst eq/bal ratios'] > 0.9) | pd.isna(df['(L)Mean of 10 worst eq/bal ratios']))
        & (df['(L)Lowest equity/balance ratio'] > 0.95)
        
        & (df['(L)Mean hours stuck'] < 1.0)
        & (df['(L)ADG per exposure'] > 0.05)

        #& (big_data['long'] == True)
        #& (big_data['adgl_%'] > 0.04)
        #& (big_data['short'] == True)
        #& (big_data['adgt_%'] > 0.1)
        #& (big_data['bnkrl_%'] > 95.0)
        #& (big_data['bnkrs_%'] > 95.0)
        #& (big_data['ebrs'] > .85)
        #& (big_data['ebrl'] > .8)
        #& (big_data['v'] == '5.5')

        # & (big_data['tg_%'] > 10)
        # & (big_data['ebr'] > .85)
        ]

    # print if df is empty
    if df.empty:
        not_found_config.append(symbol)
        continue
        
    found_config.append(symbol)
    # sort df by '(L)ADG per exposure' column in descending order
    df = df.sort_values('(L)Price action distance mean', ascending=True)
    # string replace ROOT_PATH with '' in file column
    df['file'] = df['file'].str.replace(ROOT_PATH + '/downloaded_configs/passivbot_v5.9.x/configs/', '')
    # move file and Symbol columns to the beginning
    df = df[['file', 'Symbol'] + [c for c in df.columns if c not in ['file', 'Symbol']]]

    excel_data_to_write[symbol] = df
    # get the first row from df and add it to best_configs
display(not_found_config)
display(found_config)

EXCEL_FILE_NAME = r'C:\AgodaGit\passivbot\downloaded_configs\generated_proposal.xlsx'
excel_writer = pd.ExcelWriter(EXCEL_FILE_NAME, engine='xlsxwriter')

for key, value in excel_data_to_write.items():
    display(value)
    value.to_excel(excel_writer, index=False, sheet_name=key)

excel_writer.save()
excel_writer.close()

['KAVAUSDT', 'UNFIUSDT', 'COMPUSDT', 'ETHUSDT', 'LTCUSDT']

['ATOMUSDT',
 'CHZUSDT',
 'DOGEUSDT',
 'DOTUSDT',
 'DYDXUSDT',
 'EOSUSDT',
 'HBARUSDT',
 'LINKUSDT',
 'MATICUSDT',
 'OPUSDT',
 'SANDUSDT',
 'SUSHIUSDT',
 'XRPUSDT',
 'ADAUSDT',
 'ALGOUSDT',
 'AVAXUSDT',
 'BNBUSDT',
 'ETCUSDT',
 'NEARUSDT',
 'TRXUSDT',
 'WAVESUSDT',
 'XLMUSDT',
 'XMRUSDT',
 'FTMUSDT',
 'CRVUSDT',
 'GRTUSDT',
 'GALAUSDT',
 'RUNEUSDT',
 'AAVEUSDT',
 'AGIXUSDT',
 'APEUSDT',
 'APTUSDT',
 'ARUSDT',
 'FETUSDT',
 'INJUSDT',
 'RNDRUSDT',
 'UNIUSDT']

,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
1657,older_versions/v5.9.3_clock_realistic/tests/realistic_toaster_ls_9/ATOMUSDT_20230212022827_3e196/config.json,ATOMUSDT,True,0.138,NaN,0.0344,NaN,NaN,NaN,0.830100,0.005159,0.005235,0.080659,100.0,0.9770,NaN,0.000446,0.0,0.0,NaN,18514.0,NaN,14478.0,8.0,0.792290,90.6333,bybit,5.9.3_clock_realistic,1100.0,clock,18.81,5.0,False
4343,v5.9.5/neat/july_2023/neat_july_03_0.2WE/ATOMUSDT_20230721041905_4e9ff/config.json,ATOMUSDT,True,0.225,0.1130,0.0450,0.0226,NaN,0.097359,0.047610,0.008513,0.014127,0.295183,100.0,0.9545,0.9703,0.002064,6416.0,2431.0,NaN,54.0,NaN,215.0,6419.0,0.987764,66.7667,bybit,5.9.5,640.0,neat_grid,18.81,5.0,False
5974,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_bwtpoff_v3/ATOMUSDT_20230617044621_74172/config.json,ATOMUSDT,True,0.170,0.0585,0.0340,0.0117,NaN,0.065667,0.000000,0.008842,0.009903,0.173452,100.0,0.9704,0.9738,0.001678,6719.0,5487.0,NaN,0.0,NaN,0.0,12182.0,0.599865,129.7500,bybit,5.9.5,610.0,recursive_grid,18.81,5.0,False
2833,v5.9.5/clock/july_2023/clock_85variance0.2_latest/ATOMUSDT_20230701221720_cae00/config.json,ATOMUSDT,True,0.226,0.1820,0.0451,0.0364,NaN,0.092585,0.026690,0.008974,0.010859,0.195239,100.0,0.9674,0.9762,0.001543,0.0,0.0,NaN,4151.0,NaN,629.0,11154.0,0.942419,65.6667,bybit,5.9.5,630.0,clock,18.81,5.0,False
4209,v5.9.5/neat/july_2023/neat_july_02_0.2we/ATOMUSDT_20230717074454_208a4/config.json,ATOMUSDT,True,0.199,0.1050,0.0398,0.0210,NaN,0.098099,0.064710,0.009070,0.014845,0.188697,100.0,0.9683,0.9733,0.002239,6461.0,2243.0,NaN,75.0,NaN,283.0,6461.0,0.975581,66.7167,bybit,5.9.5,630.0,neat_grid,18.81,5.0,False
2904,v5.9.5/clock/june_2023/clock_57124_1/ATOMUSDT_20230609063700_72416/config.json,ATOMUSDT,True,0.228,0.2020,0.0456,0.0404,NaN,0.073654,0.002605,0.009197,0.009901,0.225445,100.0,0.9632,0.9796,0.001152,0.0,0.0,NaN,3645.0,NaN,258.0,15375.0,0.735575,66.7667,bybit,5.9.5,590.0,clock,18.81,5.0,False
3049,v5.9.5/clock/june_2023/clock_85variance_0.2/ATOMUSDT_20230614042827_13813/config.json,ATOMUSDT,True,0.234,0.1830,0.0468,0.0366,NaN,0.095400,0.019040,0.009338,0.010120,0.135977,100.0,0.9761,0.9797,0.001414,0.0,0.0,NaN,3700.0,NaN,537.0,10417.0,0.993605,61.9333,bybit,5.9.5,610.0,clock,18.81,5.0,False
5215,v5.9.5/recursive/july_2023/recursive_85variance_0.2_bwtpon_noau_latest/ATOMUSDT_20230702205004_003ae/config.json,ATOMUSDT,True,0.253,0.0934,0.0507,0.0187,NaN,0.081390,0.000000,0.010087,0.010450,0.171906,100.0,0.9706,0.9778,0.001671,3641.0,1594.0,NaN,0.0,NaN,0.0,10276.0,0.970488,70.8667,bybit,5.9.5,630.0,recursive_grid,18.81,5.0,False
5773,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_01/ATOMUSDT_20230620071046_c0411/config.json,ATOMUSDT,True,0.285,0.1350,0.0571,0.0269,NaN,0.087288,0.000000,0.010434,0.011008,0.225173,100.0,0.9632,0.9755,0.001779,3889.0,1726.0,NaN,0.0,NaN,0.0,10581.0,0.839402,85.1667,bybit,5.9.5,570.0,recursive_grid,18.81,5.0,False
5148,v5.9.5/recursive/july_2023/recursive_85variance_0.2_bwtpoff_noau_latest_v2/ATOMUSDT_20230705063122_39062/config.json,ATOMUSDT,True,0.237,0.0839,0.0474,0.0168,NaN,0.098161,0.000000,0.011107,0.014386,0.300428,100.0,0.9537,0.9640,0.002504,3984.0,1678.0,NaN,0.0,NaN,0.0,11217.0,0.893162,93.2333,bybit,5.9.5,630.0,recursive_grid,18.81,5.0,False


,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
743,older_versions/v5.9.0/particle_swarm/recursive/cleanup_crew/lower_initial_qty_longonly_no_au/CHZUSDT_20221111172732_47a8a/config.json,CHZUSDT,True,0.195,NaN,0.0195,NaN,NaN,NaN,0.0,0.007692,0.009109,0.948654,100.0,0.9513,NaN,0.000672,9387.0,24202.0,0.0,NaN,0.0,NaN,10232.0,0.691831,89.9667,binance,5.9.0,619.708,NaN,8.77,5.0,False
6047,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_bwtpon/CHZUSDT_20230619062819_07308/config.json,CHZUSDT,True,0.241,0.195,0.0483,0.039,NaN,0.024324,0.0,0.010349,0.009013,0.290109,100.0,0.9550,0.9814,0.000778,8309.0,1763.0,NaN,0.0,NaN,0.0,11631.0,0.625207,75.2833,bybit,5.9.5,570.000,recursive_grid,8.77,5.0,False
221,older_versions/v5.9.0/harmony/old/1m/mdcl_hardcore_scalp/CHZUSDT_20220818203915_b4260/config.json,CHZUSDT,True,0.466,NaN,0.0471,NaN,0.1447,NaN,NaN,0.015262,0.015970,0.688394,100.0,0.9592,NaN,0.000990,12601.0,29143.0,79.0,NaN,537.0,NaN,14344.0,0.266999,70.5667,bybit,5.9.0,558.707,NaN,8.77,5.0,False


,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
5849,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_bwtpoff_4/DOGEUSDT_20230617213948_fcfb8/config.json,DOGEUSDT,True,0.272,0.174,0.0545,0.0348,NaN,0.040201,0.000000,0.006292,0.007752,0.180591,100.0,0.9694,0.9764,0.001325,15478.0,4518.0,NaN,0.0,NaN,0.0,23360.0,0.408335,77.08330,bybit,5.9.5,740.000,recursive_grid,76.51,5.0,False
744,older_versions/v5.9.0/particle_swarm/recursive/cleanup_crew/lower_initial_qty_longonly_no_au/DOGEUSDT_20221111172732_47a8a/config.json,DOGEUSDT,True,0.221,NaN,0.0222,NaN,NaN,NaN,0.000000,0.006373,0.009950,0.474581,100.0,0.9678,NaN,0.000308,13264.0,14214.0,0.0,NaN,0.0,NaN,14285.0,0.754078,258.68300,binance,5.9.0,814.624,NaN,76.51,5.0,False
6675,v5.9.5/recursive/june_2023/au_enabled/57125_1_recursive/DOGEUSDT_20230610084607_26fde/config.json,DOGEUSDT,True,0.298,0.208,0.0595,0.0417,NaN,0.056055,0.066640,0.006559,0.009135,0.189617,100.0,0.9681,0.9769,0.001634,9612.0,3078.0,NaN,760.0,NaN,2293.0,33225.0,0.353863,65.15000,bybit,5.9.5,720.000,recursive_grid,76.51,5.0,False
676,older_versions/v5.9.0/particle_swarm/recursive/cleanup_crew/longonly_NOAU_25maxstuck/DOGEUSDT_20221116081822_3d04a/config.json,DOGEUSDT,True,0.289,NaN,0.0290,NaN,NaN,NaN,0.000000,0.006673,0.009380,0.622910,100.0,0.9616,NaN,0.000575,35306.0,27796.0,0.0,NaN,0.0,NaN,35903.0,0.568274,283.21700,binance,5.9.0,852.624,NaN,76.51,5.0,False
4352,v5.9.5/neat/july_2023/neat_july_03_0.2WE/DOGEUSDT_20230721041905_4e9ff/config.json,DOGEUSDT,True,0.258,0.134,0.0517,0.0269,NaN,0.087214,0.082020,0.006862,0.018909,0.327528,100.0,0.9507,0.9590,0.002726,11376.0,12916.0,NaN,64.0,NaN,1815.0,11435.0,0.491568,57.78330,bybit,5.9.5,770.000,neat_grid,76.51,5.0,False
4218,v5.9.5/neat/july_2023/neat_july_02_0.2we/DOGEUSDT_20230717074454_208a4/config.json,DOGEUSDT,True,0.210,0.127,0.0420,0.0254,NaN,0.097406,0.070150,0.006957,0.014087,0.210841,100.0,0.9651,0.9691,0.002078,9047.0,9175.0,NaN,84.0,NaN,2104.0,9113.0,0.619406,64.03330,bybit,5.9.5,760.000,neat_grid,76.51,5.0,False
6050,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_bwtpon/DOGEUSDT_20230619062819_07308/config.json,DOGEUSDT,True,0.318,0.231,0.0637,0.0461,NaN,0.045139,0.000000,0.007119,0.009549,0.228745,100.0,0.9627,0.9747,0.001647,13105.0,2369.0,NaN,0.0,NaN,0.0,47512.0,0.265498,77.06670,bybit,5.9.5,700.000,recursive_grid,76.51,5.0,False
3057,v5.9.5/clock/june_2023/clock_85variance_0.2/DOGEUSDT_20230614042827_13813/config.json,DOGEUSDT,True,0.204,0.165,0.0408,0.0329,NaN,0.069340,0.040940,0.007205,0.011538,0.228782,100.0,0.9628,0.9693,0.001808,0.0,0.0,NaN,3827.0,NaN,1006.0,33364.0,0.463426,44.50000,bybit,5.9.5,740.000,clock,76.51,5.0,False
2712,v5.9.5/clock/august_2023/clock_85variance_0.2_aug/DOGEUSDT_20230804050422_49a00/config.json,DOGEUSDT,True,0.226,0.131,0.0451,0.0262,NaN,0.076304,0.024180,0.007214,0.009923,0.210453,100.0,0.9652,0.9745,0.001718,0.0,0.0,NaN,7600.0,NaN,1543.0,90717.0,0.189005,31.61670,bybit,5.9.5,790.000,clock,76.51,5.0,False
2646,v5.9.5/clock/august_2023/clock_85variance0.25/DOGEUSDT_20230812011502_1f935/config.json,DOGEUSDT,True,0.206,0.119,0.0515,0.0298,NaN,0.082709,0.023280,0.007382,0.009646,0.210097,100.0,0.9566,0.9703,0.002139,0.0,0.0,NaN,7634.0,NaN,1419.0,96008.0,0.181034,41.68330,bybit,5.9.5,790.000,clock,76.51,5.0,False


...

,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
3064,v5.9.5/clock/june_2023/clock_85variance_0.2/FETUSDT_20230614042827_13813/config.json,FETUSDT,True,0.415,0.337,0.0830,0.0674,NaN,0.076043,0.008167,0.005458,0.004951,0.073130,100.0,0.9864,0.9919,0.000635,0.0,0.0,NaN,1959.0,NaN,325.0,5945.0,0.351750,33.2667,bybit,5.9.5,120.0,clock,16.3,5.0,False
5990,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_bwtpoff_v3/FETUSDT_20230617044621_74172/config.json,FETUSDT,True,0.469,0.393,0.0938,0.0785,NaN,0.068651,0.000000,0.005570,0.004958,0.067067,100.0,0.9874,0.9914,0.000824,1820.0,2098.0,NaN,0.0,NaN,0.0,3428.0,0.403415,28.1667,bybit,5.9.5,120.0,recursive_grid,16.3,5.0,False
4359,v5.9.5/neat/july_2023/neat_july_03_0.2WE/FETUSDT_20230721041905_4e9ff/config.json,FETUSDT,True,0.209,0.240,0.0417,0.0481,NaN,0.056581,0.042950,0.006357,0.007661,0.074981,100.0,0.9861,0.9879,0.001134,2023.0,721.0,NaN,16.0,NaN,201.0,2024.0,0.743315,21.6000,bybit,5.9.5,150.0,neat_grid,16.3,5.0,False
6124,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57125_1_0.2WE_noau/FETUSDT_20230612180138_bc4bc/config.json,FETUSDT,True,0.383,0.319,0.0766,0.0639,NaN,0.039550,0.000000,0.006495,0.005225,0.069308,100.0,0.9847,0.9940,0.000486,1345.0,587.0,NaN,0.0,NaN,0.0,4040.0,0.421431,21.7833,bybit,5.9.5,110.0,recursive_grid,16.3,5.0,False
2719,v5.9.5/clock/august_2023/clock_85variance_0.2_aug/FETUSDT_20230804050422_49a00/config.json,FETUSDT,True,0.369,0.205,0.0737,0.0410,NaN,0.092956,0.071280,0.006574,0.006021,0.078433,100.0,0.9797,0.9884,0.000994,0.0,0.0,NaN,3009.0,NaN,1028.0,4875.0,0.455980,13.4000,bybit,5.9.5,170.0,clock,16.3,5.0,False
2922,v5.9.5/clock/june_2023/clock_57124_1/FETUSDT_20230609063700_72416/config.json,FETUSDT,True,0.543,0.519,0.1090,0.1040,NaN,0.088232,0.012200,0.007474,0.006431,0.047042,100.0,0.9910,0.9925,0.000729,0.0,0.0,NaN,585.0,NaN,153.0,5365.0,0.411619,24.9500,bybit,5.9.5,100.0,clock,16.3,5.0,False
6682,v5.9.5/recursive/june_2023/au_enabled/57125_1_recursive/FETUSDT_20230610084607_26fde/config.json,FETUSDT,True,0.472,0.359,0.0945,0.0719,NaN,0.086251,0.057500,0.007677,0.007246,0.106578,100.0,0.9807,0.9874,0.001222,1694.0,741.0,NaN,145.0,NaN,243.0,4764.0,0.331653,18.4667,bybit,5.9.5,110.0,recursive_grid,16.3,5.0,False
1815,v5.9.5/clock/april_2023/clock_longonly_01_04_27_23_adg10/FETUSDT_20230428025037_bc99e/config.json,FETUSDT,True,1.150,0.648,0.3450,0.1940,NaN,NaN,0.136300,0.007687,0.010465,0.089538,100.0,0.9753,0.9806,0.002539,0.0,0.0,NaN,1825.0,NaN,2236.0,256.0,0.488608,20.9500,bybit,5.9.5,88.0,clock,16.3,5.0,False
5856,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_bwtpoff_4/FETUSDT_20230617213948_fcfb8/config.json,FETUSDT,True,0.426,0.458,0.0851,0.0916,NaN,0.065457,0.000000,0.007782,0.005793,0.055099,100.0,0.9875,0.9927,0.000603,1307.0,709.0,NaN,0.0,NaN,0.0,3341.0,0.553535,32.8000,bybit,5.9.5,120.0,recursive_grid,16.3,5.0,False
6057,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_bwtpon/FETUSDT_20230619062819_07308/config.json,FETUSDT,True,0.473,0.634,0.0946,0.1270,NaN,0.062112,0.000000,0.008763,0.007006,0.060940,100.0,0.9885,0.9941,0.000607,471.0,209.0,NaN,0.0,NaN,0.0,2092.0,0.691850,30.8167,bybit,5.9.5,81.0,recursive_grid,16.3,5.0,False


,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
3449,v5.9.5/clock/march_2023/clock_longonly_03_25_23/INJUSDT_20230325173045_226b9/config.json,INJUSDT,True,0.335,0.777,0.1000,0.2330,NaN,NaN,0.296500,0.005917,0.005127,0.056237,100.0,0.9791,0.9909,0.000834,0.0,0.0,NaN,4621.0,NaN,2772.0,65.0,0.704430,7.23333,bybit,5.9.5,220.0,clock,34.07,5.0,False
6132,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57125_1_0.2WE_noau/INJUSDT_20230612180138_bc4bc/config.json,INJUSDT,True,0.226,0.241,0.0452,0.0483,NaN,0.028794,0.000000,0.007092,0.007697,0.079875,100.0,0.9852,0.9866,0.000983,1561.0,940.0,NaN,0.0,NaN,0.0,7252.0,0.691662,34.30000,bybit,5.9.5,280.0,recursive_grid,34.07,5.0,False
6397,v5.9.5/recursive/june_2023/au_disabled/treasure_chest/57124_bwtpon_latest/INJUSDT_20230623000751_a9eb2/config.json,INJUSDT,True,0.243,0.406,0.0486,0.0812,NaN,0.034710,0.000000,0.007638,0.007811,0.105725,100.0,0.9808,0.9830,0.001191,4818.0,1690.0,NaN,0.0,NaN,0.0,11884.0,0.360273,43.23330,bybit,5.9.5,280.0,recursive_grid,34.07,5.0,False
6690,v5.9.5/recursive/june_2023/au_enabled/57125_1_recursive/INJUSDT_20230610084607_26fde/config.json,INJUSDT,True,0.208,0.275,0.0415,0.0550,NaN,0.041976,0.117000,0.007834,0.009815,0.093210,100.0,0.9829,0.9849,0.001350,1655.0,858.0,NaN,398.0,NaN,519.0,6656.0,0.667799,55.71670,bybit,5.9.5,280.0,recursive_grid,34.07,5.0,False
2530,v5.9.5/clock/april_2023/clock_longonly_04_06_23/INJUSDT_20230406223306_5b16b/config.json,INJUSDT,True,0.646,0.890,0.1940,0.2670,NaN,NaN,0.298700,0.008247,0.012594,0.141174,100.0,0.9629,0.9777,0.003575,0.0,0.0,NaN,4036.0,NaN,4050.0,523.0,0.643582,8.06667,bybit,5.9.5,230.0,clock,34.07,5.0,False
5998,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_bwtpoff_v3/INJUSDT_20230617044621_74172/config.json,INJUSDT,True,0.197,0.224,0.0395,0.0447,NaN,0.052930,0.000000,0.008428,0.012819,0.181102,100.0,0.9693,0.9720,0.002296,3438.0,4718.0,NaN,0.0,NaN,0.0,9154.0,0.411565,35.81670,bybit,5.9.5,300.0,recursive_grid,34.07,5.0,False
5797,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3_01/INJUSDT_20230620071046_c0411/config.json,INJUSDT,True,0.317,0.469,0.0634,0.0938,NaN,0.054552,0.000000,0.009092,0.007304,0.074574,100.0,0.9822,0.9879,0.000916,1692.0,1355.0,NaN,0.0,NaN,0.0,6502.0,0.644402,79.33330,bybit,5.9.5,260.0,recursive_grid,34.07,5.0,False
5698,v5.9.5/recursive/june_2023/au_disabled/57124_3_bwtpon_v1/INJUSDT_20230627233134_ab098/config.json,INJUSDT,True,0.206,0.237,0.0412,0.0474,NaN,0.047087,0.000000,0.009503,0.007635,0.092202,100.0,0.9831,0.9860,0.000951,2240.0,2022.0,NaN,0.0,NaN,0.0,7512.0,0.636722,69.66670,bybit,5.9.5,310.0,recursive_grid,34.07,5.0,False
2932,v5.9.5/clock/june_2023/clock_57124_1/INJUSDT_20230609063700_72416/config.json,INJUSDT,True,0.345,0.402,0.0691,0.0805,NaN,0.092748,0.008777,0.009503,0.009888,0.104734,100.0,0.9810,0.9824,0.001525,0.0,0.0,NaN,2490.0,NaN,721.0,24345.0,0.244170,33.95000,bybit,5.9.5,280.0,clock,34.07,5.0,False
6065,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_bwtpon/INJUSDT_20230619062819_07308/config.json,INJUSDT,True,0.303,0.406,0.0605,0.0812,NaN,0.066714,0.000000,0.010041,0.015820,0.311141,100.0,0.9525,0.9619,0.002759,2139.0,1914.0,NaN,0.0,NaN,0.0,8253.0,0.499600,72.30000,bybit,5.9.5,260.0,recursive_grid,34.07,5.0,False


,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
2099,v5.9.5/clock/april_2023/clock_longonly_03_04_30_23_adg10/RNDRUSDT_20230430081243_ad5a8/config.json,RNDRUSDT,True,0.980,0.911,0.2940,0.2730,NaN,NaN,0.29890,0.004933,0.006434,0.089542,100.0,0.9753,0.9846,0.001568,0.0,0.0,NaN,2313.0,NaN,907.0,224.0,0.445372,12.16670,bybit,5.9.5,64.0,clock,8.03,5.0,False
3643,v5.9.5/clock/may_2023/clock_longonly_01_may_25_23/RNDRUSDT_20230525165830_f028f/config.json,RNDRUSDT,True,0.707,0.549,0.2120,0.1650,NaN,NaN,0.13300,0.006110,0.006177,0.061242,100.0,0.9827,0.9869,0.001064,0.0,0.0,NaN,1642.0,NaN,1055.0,52.0,0.967194,23.31670,bybit,5.9.5,110.0,clock,8.03,5.0,False
3701,v5.9.5/clock/may_2023/clock_longonly_02_may_13_23/RNDRUSDT_20230513164546_acfb3/config.json,RNDRUSDT,True,0.650,0.780,0.1950,0.2340,NaN,NaN,0.12970,0.006199,0.006170,0.059792,100.0,0.9831,0.9873,0.001089,0.0,0.0,NaN,1345.0,NaN,955.0,112.0,0.973096,46.91670,bybit,5.9.5,98.0,clock,8.03,5.0,False
2608,v5.9.5/clock/april_2023/clock_longonly_04_09_23/RNDRUSDT_20230409075417_0917b/config.json,RNDRUSDT,True,0.904,0.552,0.2710,0.1660,NaN,NaN,0.26720,0.006387,0.007047,0.098227,100.0,0.9732,0.9926,0.001723,0.0,0.0,NaN,1840.0,NaN,777.0,119.0,0.560262,3.76667,bybit,5.9.5,64.0,clock,8.03,5.0,False
1835,v5.9.5/clock/april_2023/clock_longonly_01_04_27_23_adg10/RNDRUSDT_20230428025037_bc99e/config.json,RNDRUSDT,True,1.180,1.210,0.3550,0.3640,NaN,NaN,0.27790,0.006581,0.009392,0.104317,100.0,0.9738,0.9812,0.002178,0.0,0.0,NaN,1628.0,NaN,980.0,179.0,0.610463,8.81667,bybit,5.9.5,71.0,clock,8.03,5.0,False
2345,v5.9.5/clock/april_2023/clock_longonly_04_04_20_23/RNDRUSDT_20230420060455_9ead8/config.json,RNDRUSDT,True,0.807,0.889,0.2420,0.2670,NaN,NaN,0.29430,0.006748,0.007807,0.083084,100.0,0.9770,0.9939,0.002037,0.0,0.0,NaN,2646.0,NaN,1482.0,43.0,0.430719,7.35000,bybit,5.9.5,75.0,clock,8.03,5.0,False
3768,v5.9.5/clock/may_2023/clock_longonly_03_may_14_23/RNDRUSDT_20230514162802_89185/config.json,RNDRUSDT,True,0.614,0.634,0.1840,0.1900,NaN,NaN,0.14770,0.006988,0.006143,0.078621,100.0,0.9781,0.9859,0.001159,0.0,0.0,NaN,2662.0,NaN,1182.0,375.0,0.551067,6.23333,bybit,5.9.5,97.0,clock,8.03,5.0,False
3835,v5.9.5/clock/may_2023/clock_longonly_04_may_15_23/RNDRUSDT_20230515175620_cd2e0/config.json,RNDRUSDT,True,0.732,0.763,0.2200,0.2290,NaN,NaN,0.14780,0.007260,0.006323,0.078821,100.0,0.9781,0.9856,0.001372,0.0,0.0,NaN,2688.0,NaN,1651.0,65.0,0.527894,6.65000,bybit,5.9.5,97.0,clock,8.03,5.0,False
1769,v5.9.5/clock/april_2023/clock_longonly_01_04_16_23/RNDRUSDT_20230416212842_7374b/config.json,RNDRUSDT,True,0.908,1.030,0.2720,0.3080,NaN,NaN,0.29440,0.007554,0.009740,0.093169,100.0,0.9744,0.9916,0.002767,0.0,0.0,NaN,2762.0,NaN,1688.0,93.0,0.374541,8.25000,bybit,5.9.5,71.0,clock,8.03,5.0,False
1901,v5.9.5/clock/april_2023/clock_longonly_02_04_17_23/RNDRUSDT_20230417213250_71e6f/config.json,RNDRUSDT,True,0.968,0.819,0.2900,0.2460,NaN,NaN,0.26020,0.007623,0.009022,0.092275,100.0,0.9747,0.9918,0.002479,0.0,0.0,NaN,2046.0,NaN,1023.0,376.0,0.500489,14.25000,bybit,5.9.5,72.0,clock,8.03,5.0,False


,file,Symbol,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Average daily gain,(L)Average daily gain weighted,(L)DG mean std ratio,(L)Exposure ratios mean,(L)Loss to profit ratio,(L)Price action distance mean,(L)Price action distance std,(L)Price action distance max,(L)Closest bankruptcy,(L)Lowest equity/balance ratio,(L)Mean of 10 worst eq/bal ratios,(L)Equity/balance ratio std,(L)No. initial entries,(L)No. reentries,(L)No. unstuck entries,(L)No. unstuck/EMA entries,(L)No. unstuck closes,(L)No. unstuck/EMA closes,(L)No. normal closes,(L)Mean hours stuck,(L)Max hours stuck,Exchange,version,No. days,Passivbot mode,volume,min_notional,live
5760,v5.9.5/recursive/june_2023/au_disabled/57124_tests/57124_3/UNIUSDT_20230602220832_05aea/config.json,UNIUSDT,True,0.2320,0.1020,0.0464,0.02040,NaN,NaN,0.000000,0.010620,0.011084,0.308800,100.0,0.9526,0.9603,0.001675,16965.0,3585.0,NaN,0.0,NaN,0.0,16772.0,0.633168,82.7333,bybit,5.9.5,990.0,recursive_grid,9.16,5.0,False
2751,v5.9.5/clock/august_2023/clock_85variance_0.2_aug/UNIUSDT_20230804050422_49a00/config.json,UNIUSDT,True,0.1280,0.1050,0.0255,0.02100,NaN,0.072284,0.065010,0.011341,0.014327,0.248147,100.0,0.9604,0.9660,0.001991,0.0,0.0,NaN,5821.0,NaN,1358.0,14028.0,0.976089,47.7667,bybit,5.9.5,860.0,clock,9.16,5.0,False
5263,v5.9.5/recursive/july_2023/recursive_85variance_0.2_bwtpon_noau_latest/UNIUSDT_20230702205004_003ae/config.json,UNIUSDT,True,0.1160,0.0492,0.0232,0.00983,NaN,0.053392,0.000000,0.011625,0.014097,0.299480,100.0,0.9536,0.9592,0.002332,12795.0,2232.0,NaN,0.0,NaN,0.0,12685.0,0.722606,77.7000,bybit,5.9.5,830.0,recursive_grid,9.16,5.0,False
2685,v5.9.5/clock/august_2023/clock_85variance0.25/UNIUSDT_20230812011502_1f935/config.json,UNIUSDT,True,0.1530,0.1080,0.0382,0.02710,NaN,0.077333,0.039710,0.011782,0.017781,0.242815,100.0,0.9512,0.9562,0.003221,0.0,0.0,NaN,4871.0,NaN,902.0,15716.0,0.969938,47.7333,bybit,5.9.5,870.0,clock,9.16,5.0,False
4391,v5.9.5/neat/july_2023/neat_july_03_0.2WE/UNIUSDT_20230721041905_4e9ff/config.json,UNIUSDT,True,0.1140,0.0605,0.0227,0.01210,NaN,0.082568,0.102800,0.012045,0.017958,0.226244,100.0,0.9631,0.9660,0.002362,7490.0,9136.0,NaN,42.0,NaN,126.0,7504.0,0.836268,90.5167,bybit,5.9.5,850.0,neat_grid,9.16,5.0,False
5129,v5.9.5/recursive/july_2023/recursive_85variance_0.2_bwtpoff_noau_latest/UNIUSDT_20230703230023_8779d/config.json,UNIUSDT,True,0.0891,0.0442,0.0178,0.00884,NaN,0.035736,0.000000,0.013200,0.009648,0.164443,100.0,0.9715,0.9779,0.001013,8923.0,3368.0,NaN,0.0,NaN,0.0,12373.0,0.812482,94.4833,bybit,5.9.5,840.0,recursive_grid,9.16,5.0,False
5454,v5.9.5/recursive/july_2023/recursive_july_03_auenabled_0.2WE/UNIUSDT_20230714162727_dfc69/config.json,UNIUSDT,True,0.1540,0.0717,0.0308,0.01430,NaN,0.068498,0.049360,0.013788,0.015141,0.319828,100.0,0.9514,0.9618,0.002046,4102.0,1489.0,NaN,115.0,NaN,237.0,22212.0,0.720572,64.9667,bybit,5.9.5,850.0,recursive_grid,9.16,5.0,False
2961,v5.9.5/clock/june_2023/clock_57124_1/UNIUSDT_20230608172310_18d9f/config.json,UNIUSDT,True,0.1030,0.1270,0.0206,0.02550,NaN,0.066888,0.009065,0.015562,0.016264,0.176756,100.0,0.9680,0.9793,0.001050,0.0,0.0,NaN,3637.0,NaN,495.0,18772.0,0.835677,65.7833,bybit,5.9.5,800.0,clock,9.16,5.0,False


C:\Users\skhan\.virtualenvs\passivbot\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
-- whatevercrash --
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
result_analysis = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
result_analysis.to_csv('5.8.csv')
result_analysis

In [ ]:
result_analysis.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = result_analysis[['file', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl', 'pastdl',
         'maxss', 'pamaxs','pastds']]
dx